In [1]:
# Importing necessary libraries for the project
import numpy as np
import pandas as pd

from sklearn.base import ClassifierMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline, make_pipeline as base_make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier

from matplotlib import pyplot as plt
from tabulate import tabulate

import calendar
from datetime import datetime, timedelta

In [2]:
# Constants for the telemarketing project
from pathlib import Path

CATEGORICAL_FEATURES = [
    "contact",
    "day_of_week",
    "default",
    "education",
    "housing",
    "job",
    "loan",
    "marital",
    "month",
    "poutcome",
    "year",
]
NUMERICAL_FEATURES = [
    "age",
    "campaign",
    "pdays",
    "previous",
    "emp.var.rate",
    "cons.price.idx",
    "cons.conf.idx",
    "euribor3m",
    "nr.employed",
]
BINARY_FEATURES = [
    "y",
]

DATA_DIR = Path("data")
RAW_DATA_DIR = DATA_DIR / "raw"
INTERIM_DATA_DIR = DATA_DIR / "interim"
PROCESSED_DATA_DIR = DATA_DIR / "processed"

DATA_FILENAME = "bank-additional-full.csv"
APPROACHED_DATA_FILENAME = "approached_data.csv"
NOT_APPROACHED_DATA_FILENAME = "not_approached_data.csv"

HONOLULU_BLUE = "#1F77B4"
IMPERIAL_RED = "#F0534F"
PERSIAN_GREEN = "#27A69A"

### Time-based Data Split Strategy

- **Test period selection**: 2010 data serves as the test set to ensure realistic and future-proof evaluation

- **Training and validation periods**: 2008-2009 data is used for model training and validation phases

- **Temporal integrity**: The time-based split of training and test datasets maintains chronological order, preventing data leakage where future information inappropriately influences model training

- **Generalization assessment**: This approach enables honest evaluation of how effectively the model performs on completely unseen data

- **Business application**: The split structure supports realistic business strategy simulations that mirror actual real-world deployment conditions

**note**

We deliberately avoided stratifying the data splits by the target variable. Stratification requires random shuffling, which is fundamentally incompatible with the strict chronological ordering necessary for a time-series problem.
Our priority is to prevent data leakage and create a realistic train/test split that respects the arrow of time. The class imbalance is instead addressed at the model training stage using the class_weight parameter. This method correctly handles the imbalance without compromising the temporal integrity of our validation framework.

In [3]:
# Load not_approached dataset
df = pd.read_csv(PROCESSED_DATA_DIR / NOT_APPROACHED_DATA_FILENAME)

FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/not_approached_data.csv'

In [ ]:
# Define X and y for modeling
X = df.drop(columns=["y"], axis=1)
y = df["y"]

print(f"X shape: {X.shape}, y shape: {y.shape}")

X.head()

In [ ]:
# dertemine the split in train and test data.
print(X.groupby(['year']).size().sort_index())
# Ratio between 2010 data and the rest
print(f"Ratio of 2010 data to the rest: {X[X['year'] == 2010].shape[0] / X[X['year'] != 2010].shape[0]:.2f}")

Our time-based data split revealed a key challenge: the most recent data available for testing (2010) is significantly smaller than the preceding training years. Standard evaluation on this set alone would be statistically unreliable.

Therefore, we have adopted a more sophisticated validation protocol to ensure trustworthiness:

### Model selection will be driven by a stable, averaged score from a rigorous time-series cross-validation on the large 2008-2009 dataset

**The Problem with unequal spaced time series:**

TimeSeriesSplit splits by row count, not actual time duration. This means:
1. **Unequal Spacing:** Test sets (same number of rows) can cover vastly different **time durations**.
2. **Imbalanced Periods:** Different test sets will represent different years/contexts (e.g., 2008 data vs. 2009 data).

**Result:** Cross-validation metrics (RMSE, MAE) are **not truly comparable** across folds, as they reflect performance over different time horizons and contexts.

**How to Address It:**
1. **Keep TimeSeriesSplit**: It's still correct for chronological splits.
2. **Smart Feature Engineering (Key!)**:
   * **Time-based Features:** Add year, month, day_of_week, etc., so the model knows the temporal context.
   * **Gap Features:** Add days_since_last_observation or observations_in_last_X_days to inform the model about data density.
3. **Careful Evaluation**:
   * Report **metrics for each individual fold** to see how performance shifts over time/contexts.
   * **Contextualize** results; explain performance changes based on the different periods covered by each fold


### The small 2010 test set will then serve as a final, mandatory sanity check, with its performance explicitly framed by confidence intervals to account for potential variance.

**References need to be checked**

Statistical Issues with Small Test Sets
This issue is well-documented in statistics and machine learning:

Reference: Hastie, T., Tibshirani, R., & Friedman, J. (2009). The Elements of Statistical Learning (2nd ed.). Springer.
Chapter 7 discusses model assessment and the impact of sample size on estimation stability. Small test sets lead to high-variance estimates, necessitating robust evaluation methods like cross-validation or confidence intervals.


Reference: Kohavi, R. (1995). A Study of Cross-Validation and Bootstrap for Accuracy Estimation and Model Selection. IJCAI.
Kohavi emphasizes that small test sets produce unreliable performance estimates and recommends techniques like cross-validation or confidence intervals to quantify uncertainty.




### Concept drift and temporal imbalance

Our dataset is influenced by two interacting temporal phenomena. First, concept drift, which could be driven by for example the 2008 financial crisis, has rendered older data less predictive of current outcomes. Second, we observe a temporal instance imbalance, where this outdated 2008-era data is overrepresented in volume. While each issue is problematic on its own, their combination is especially harmful: the volume imbalance significantly amplifies the negative impact of the drift. To address this, we apply time-based sample weighting — not to equalize instance counts, but to rebalance the influence of different time periods. This ensures the model prioritizes learning from more recent, and therefore more relevant, observations."


### Feature transformation 

Some distributions have a heavy tail. In order to quickly check this the describe function can be used. 

In [ ]:
# Loading not approached dataset
not_approached = pd.read_csv(PROCESSED_DATA_DIR / NOT_APPROACHED_DATA_FILENAME)

In [ ]:
# Decribe the dataset
not_approached.describe()

Age: Is slightly skewed and can be handled with standard scaler.

Campaign: Is serverly skewed and problamatic for standardization. Max value is 56 while 75%Q is 3 and the mean is 2.567. Apply power transformation like log.

emp.var.rate: Does not have any signs of skew

cons.price.idx: Does not have any signs of skew

cons.conf.idx: Does not have any signs of skew

euribor3m: Does not have any signs of skew

nr.empployed: Does not have any signs of skew

year: Does not have any signs of skew

y: target variable

In [ ]:
# Load approached dataset
approached = pd.read_csv(PROCESSED_DATA_DIR / APPROACHED_DATA_FILENAME)
approached.shape

In [ ]:
# decribe features pdays and previous
approached.describe()

In [ ]:
# create distribution plots for pdays and previous
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
approached["pdays"].hist(bins=50, ax=axes[0], color=HONOLULU_BLUE)
axes[0].set_title("Distribution of Pdays")
axes[0].set_xlabel("Pdays")
axes[0].set_ylabel("Frequency")
approached["previous"].hist(bins=50, ax=axes[1], color=HONOLULU_BLUE)
axes[1].set_title("Distribution of Previous")
axes[1].set_xlabel("Previous")
axes[1].set_ylabel("Frequency")
plt.tight_layout()
plt.show()

** Analysis of appoached values

Age: Is slightly skewed and can be handled with standard scaler.

Campaign: Is serverly skewed and problamatic for standardization. Max value is 13 while 75%Q is 2 and the mean is 1.82. Apply power transformation like log.

pdays: Moderate skew, log transformation advised

previous: Strong skew, log transformation advised

emp.var.rate: Does not have any signs of skew

cons.price.idx: Does not have any signs of skew

cons.conf.idx: Does not have any signs of skew

euribor3m: Does not have any signs of skew

nr.empployed: Does not have any signs of skew

year: Does not have any signs of skew

y: target variable

In [ ]:
# apply log transformation to pdays and plot the distribution
approached["log_pdays"] = np.log1p(approached["pdays"])
fig, ax = plt.subplots(figsize=(8, 5))
approached["log_pdays"].hist(bins=50, ax=ax, color=HONOLULU_BLUE)
ax.set_title("Log Transformation of Pdays") 
ax.set_xlabel("Log(Pdays)")
ax.set_ylabel("Frequency") 
plt.show()

In [ ]:
# apply log transformation to previous and plot the distribution
approached["log_previous"] = np.log1p(approached["previous"])
fig, ax = plt.subplots(figsize=(8, 5))
approached["log_previous"].hist(bins=50, ax=ax, color=HONOLULU_BLUE)
ax.set_title("Log Transformation of Previous")
ax.set_xlabel("Log(Previous)")
ax.set_ylabel("Frequency")
plt.show()

In [ ]:
# describe the log transformed features
approached[["log_pdays", "log_previous"]].describe()

The log transformation of previous did not normalize the data. However it was a good step in the right direction since extreme values are limited.

### Performance metrics

**Core metric**

The project aims to create stakeholder value by maximizing the profit of the telemarketing department. Profit maximalization based ont graph??

- Expected Profit = (Number of True Positives * Revenue_per_sale) - (Number of Calls Made * Cost_per_call)

additionally



**Model calibration**

Calabritaion is essential since the strategic eval module depends on probabilities. Why needed:
- Meaningfull thresholds: if the probabilities are not calibrated there is no meaning in 'call all customers with  >0.5 probability. If the model is overconfident, the actual chance of success might be way lower.

- Accurrate profit predictions: The profit simulations relies on the predicted probabilities. Incorrect probabilities will result in incorrect profit estimations.

- Trust: For the stakeholder to make a good data driven decision they need to trust the numbers the model generates.

How to assess calibration (to do)

- Reliability diagram (Calibration curve)
- Brier score


Notities:

Wat is de winst op basis van de confusion matrix. Kosten en baten.

Het beste model is het model dat de beste winst realiseerd. Train beste model.

Winst per gebelde persoon, om test en train set goed te kunnen vergelijken.

Voor stakeholders, hoofdzaken van het onderzoek. Dus niet te technisch. 

In [ ]:
# the mean of calls during this campaign for the not approached dataset
mean_calls = not_approached["campaign"].mean() 
print(f"Mean calls during this campaign for not approached dataset: {mean_calls:.2f} times")
# the average duration of calls for the not approached dataset
# needs to be claculated for the not approached set but column is dropped
# quick peak in the original dataset leads to a mean of 258 seconds
mean_duration_minutes = 258 / 60
print(f"Mean duration of calls for not approached dataset: {mean_duration_minutes:.2f} minutes")
# average time spent per customer
average_time_on_phone_per_customer = mean_duration_minutes * mean_calls
print(f"Average time spent per customer on phone: {average_time_on_phone_per_customer:.2f} minutes")
# including the start and end of the call including the time to prepare and follow up
# preparation and follow up is estimated to be 3 minutes per call
prep_time_per_call = 3
total_prep_time = prep_time_per_call * mean_calls
average_time_per_customer = average_time_on_phone_per_customer + total_prep_time
print(f"Average time spent per customer including preparation and follow up: {average_time_per_customer:.2f} minutes")

In [ ]:
# calculate the current profit for the not approached dataset for a selected year
selected_year = 2010
hourly_wage = 35  # in euro

subset = not_approached[not_approached["year"] == selected_year]

cost_per_contact = hourly_wage * (average_time_per_customer / 60)
profit_per_success = 200

total_costs = subset.shape[0] * cost_per_contact
profit = subset[subset["y"] == 1].shape[0] * profit_per_success

total_profit = profit - total_costs
print(f"Cost per contact: {cost_per_contact:.2f} euro")
formatted_profit = f"{total_profit:,.0f}".replace(",", ".")
print(f"Total profit for year {selected_year}: {formatted_profit} euro")
